## Label Propagation Project

## Karate Social Network Implementation

In [1]:
!scalac --version

Scala compiler version 2.13.5 -- Copyright 2002-2020, LAMP/EPFL and Lightbend, Inc.



In [2]:
import scala.util.Random
import org.apache.spark.graphx.EdgeTriplet
import org.apache.spark.graphx.Graph
import org.apache.spark.graphx.VertexId
import org.apache.spark.sql.expressions.Window
import org.apache.spark.rdd.RDD
import org.apache.spark.graphx.EdgeDirection
import org.apache.spark.graphx.Edge
import org.apache.spark.sql.Row
import java.util.concurrent.atomic.AtomicInteger

Intitializing Scala interpreter ...

Spark Web UI available at http://thibaults-mbp:4040
SparkContext available as 'sc' (version = 3.1.1, master = local[*], app id = local-1619087686562)
SparkSession available as 'spark'


import scala.util.Random
import org.apache.spark.graphx.EdgeTriplet
import org.apache.spark.graphx.Graph
import org.apache.spark.graphx.VertexId
import org.apache.spark.sql.expressions.Window
import org.apache.spark.rdd.RDD
import org.apache.spark.graphx.EdgeDirection
import org.apache.spark.graphx.Edge
import org.apache.spark.sql.Row
import java.util.concurrent.atomic.AtomicInteger


In [3]:
// Defining Vertex class in order to take into account labels
class LPVertex(
    var degree : Int = 0,
    var initialLabel : Boolean = false,
    // string label name is associated with its probability
    var vertexLabels : Map[String, Double] = Map[String, Double]() 
) extends java.io.Serializable {     
    
    def setDegree(degree: Int) = {
        this.degree = degree
    }
    
   // Overriding tostring method
    override def toString() : String = {
    return "initialLabel : " + initialLabel + "/ vertexLabels = " + 
        vertexLabels.foreach { case (key, values) => println("key " + key + " - " + values )}
    }
}

defined class LPVertex


In [47]:
// Defining function to create half of vertices that are labeled
val rand = new Random(4)

def random_label(random: Random): Int = {
    random.nextInt(3)
}

def transform(VD: Int): LPVertex = {
    val label = random_label(rand);
    var v = new LPVertex();

    if (rand.nextInt(2) == 1) {
        if (label == 0) {
            v.initialLabel = true;
            v.vertexLabels = Map("1" -> 1.0, "2" -> 0.0, "3" -> 0.0);
        } else if (label == 1) {
            v.initialLabel = true;
            v.vertexLabels =  Map("1" -> 0.0, "2" -> 1.0, "3" -> 0.0);
        } else if (label == 2) {
            v.initialLabel = true;
            v.vertexLabels = Map("1" -> 0.0, "2" -> 0.0, "3" -> 1.0);
        }
    }
    v;
}

rand: scala.util.Random = scala.util.Random@1e5d773d
random_label: (random: scala.util.Random)Int
transform: (VD: Int)LPVertex


In [25]:
val count = new AtomicInteger()

count: java.util.concurrent.atomic.AtomicInteger = 0


In [26]:
def transform2(VD: Int): LPVertex = {
    var v = new LPVertex();
    if (count.get() == 27) {
        v.initialLabel = true;
        v.vertexLabels = Map("1" -> 1.0, "2" -> 0.0);
    } else if (count.get() == 17) {
        v.initialLabel = true;
        v.vertexLabels =  Map("1" -> 0.0, "2" -> 1.0);
    }
    count.incrementAndGet()
    v;
}

transform2: (VD: Int)LPVertex


In [27]:
// Vertex Progam for Pregel
def VertexProgram(id : VertexId, 
                  VD : LPVertex, 
                  m : Map[String, Double]) : LPVertex  = {
    var newLPVertex = new LPVertex()
    newLPVertex.degree = VD.degree
    newLPVertex.initialLabel = VD.initialLabel
    newLPVertex.vertexLabels = VD.vertexLabels
    
    if(VD.initialLabel == false){
        newLPVertex.vertexLabels = m.map(x => (x._1, x._2 / VD.degree))
    }
    newLPVertex
}

VertexProgram: (id: org.apache.spark.graphx.VertexId, VD: LPVertex, m: Map[String,Double])LPVertex


In [28]:
// Vertex Progam for Pregel where initial nodes change labels too after certain number of message passing
val vertex_count = new AtomicInteger()

def VertexProgram2(id : VertexId, 
                  VD : LPVertex, 
                  m : Map[String, Double]) : LPVertex  = {
    
    var newLPVertex = new LPVertex()
    vertex_count.incrementAndGet()
    newLPVertex.degree = VD.degree
    newLPVertex.initialLabel = VD.initialLabel
    newLPVertex.vertexLabels = VD.vertexLabels

    if(VD.initialLabel == false ){
        newLPVertex.vertexLabels = m.map(x => (x._1, x._2 / VD.degree))
    }
    else if (vertex_count.get() > 10){
        newLPVertex.vertexLabels = m.map(x => (x._1, x._2 / VD.degree))
    }
    newLPVertex
}

vertex_count: java.util.concurrent.atomic.AtomicInteger = 0
VertexProgram2: (id: org.apache.spark.graphx.VertexId, VD: LPVertex, m: Map[String,Double])LPVertex


In [29]:
// Send Message
def SendMsg(triplet : EdgeTriplet[LPVertex, Int]): Iterator[(VertexId, Map[String, Double])] = {
    Iterator((triplet.dstId, triplet.srcAttr.vertexLabels))
}

SendMsg: (triplet: org.apache.spark.graphx.EdgeTriplet[LPVertex,Int])Iterator[(org.apache.spark.graphx.VertexId, Map[String,Double])]


In [30]:
// Merge Message
def MergeMsg(m1 : Map[String, Double],
             m2 : Map[String, Double]) : Map[String, Double] = {
    val mergeMsg = (m1.keySet ++ m2.keySet).map {i=> (i,m1.getOrElse(i,0.0) + m2.getOrElse(i,0.0))}.toMap
    mergeMsg;
}

MergeMsg: (m1: Map[String,Double], m2: Map[String,Double])Map[String,Double]


In [31]:
val m1 :Map[String, Double] = Map("1" -> 1, "2" -> 2, "3" -> 3.0 )
val m2 :Map[String, Double] = Map( "1"-> 0.5, "2"-> 0.7, "3" -> 0.3)

m1: Map[String,Double] = Map(1 -> 1.0, 2 -> 2.0, 3 -> 3.0)
m2: Map[String,Double] = Map(1 -> 0.5, 2 -> 0.7, 3 -> 0.3)


In [32]:
MergeMsg(m1, m2)

res5: Map[String,Double] = Map(1 -> 1.5, 2 -> 2.7, 3 -> 3.3)


# Karate Social Network 

In [33]:
// Loading file
val karate_file = sc.textFile("soc-karate/soc-karate.mtx")

karate_file: org.apache.spark.rdd.RDD[String] = soc-karate/soc-karate.mtx MapPartitionsRDD[1780] at textFile at <console>:36


In [34]:
var karate_data = karate_file.toDF.withColumn("id",monotonicallyIncreasingId)

karate_data = karate_data.withColumn("rank", row_number().over(Window.orderBy("id")))
karate_data = karate_data.filter(karate_data("rank")>24)
karate_data = karate_data.drop("id","rank")

val karate_data_splitted = karate_data.withColumn("_tmp", split($"value", "\\ ")).select(
  $"_tmp".getItem(0).as("srcId"),
  $"_tmp".getItem(1).as("dstID"),
)

karate_data: org.apache.spark.sql.DataFrame = [value: string]
karate_data: org.apache.spark.sql.DataFrame = [value: string]
karate_data: org.apache.spark.sql.DataFrame = [value: string]
karate_data: org.apache.spark.sql.DataFrame = [value: string]
karate_data_splitted: org.apache.spark.sql.DataFrame = [srcId: string, dstID: string]


In [35]:
val rows: RDD[Row] = karate_data_splitted.rdd
var karate_edges = rows.map{ case Row(src:String, dist : String) => Edge(src.toLong, dist.toLong, 1)}

rows: org.apache.spark.rdd.RDD[org.apache.spark.sql.Row] = MapPartitionsRDD[1790] at rdd at <console>:38
karate_edges: org.apache.spark.rdd.RDD[org.apache.spark.graphx.Edge[Int]] = MapPartitionsRDD[1791] at map at <console>:39


In [45]:
// Creating graph
val count = new AtomicInteger()
val initialGraph = Graph.fromEdges(karate_edges, 0)

val graph = initialGraph.mapVertices({ case (id, attr) => transform((attr))})

val vertex_rdd = graph.degrees.zip(graph.vertices).map({ case (vDeg, vAttr) => vAttr._2.setDegree(vDeg._2) ; vAttr})
val edge_rdd = graph.edges

val graph2 = Graph(vertex_rdd, edge_rdd)

// We have an undirected graph
val finalGraph = Graph(graph2.vertices, graph2.edges.union(graph2.edges.reverse))

count: java.util.concurrent.atomic.AtomicInteger = 0
initialGraph: org.apache.spark.graphx.Graph[Int,Int] = org.apache.spark.graphx.impl.GraphImpl@145932a1
graph: org.apache.spark.graphx.Graph[LPVertex,Int] = org.apache.spark.graphx.impl.GraphImpl@14aef092
vertex_rdd: org.apache.spark.rdd.RDD[(org.apache.spark.graphx.VertexId, LPVertex)] = MapPartitionsRDD[3620] at map at <console>:50
edge_rdd: org.apache.spark.graphx.EdgeRDD[Int] = EdgeRDDImpl[3614] at RDD at EdgeRDD.scala:41
graph2: org.apache.spark.graphx.Graph[LPVertex,Int] = org.apache.spark.graphx.impl.GraphImpl@667ed400
finalGraph: org.apache.spark.graphx.Graph[LPVertex,Int] = org.apache.spark.graphx.impl.GraphImpl@164af661


In [46]:
finalGraph.vertices.foreach(v => println(v._1, v._2.degree, v._2.initialLabel, v._2.vertexLabels.mkString(" ")))

0000000000000000000000000000000000(13,2,true,1 -> 1.0 2 -> 0.0 3 -> 0.0)
(19,2,false,)
(34,17,false,)
(5,3,true,1 -> 0.0 2 -> 1.0 3 -> 0.0)
(15,2,false,)
(4,6,true,1 -> 0.0 2 -> 0.0 3 -> 1.0)
(21,2,true,1 -> 0.0 2 -> 0.0 3 -> 1.0)
(16,2,false,)
(22,2,true,1 -> 0.0 2 -> 1.0 3 -> 0.0)
(25,3,false,)
(28,4,true,1 -> 0.0 2 -> 0.0 3 -> 1.0)
(29,3,true,1 -> 0.0 2 -> 1.0 3 -> 0.0)
(11,3,false,)
(14,5,false,)
(32,6,false,)
(30,4,true,1 -> 0.0 2 -> 1.0 3 -> 0.0)
(24,5,false,)
(27,2,false,)
(33,12,false,)
(23,2,true,1 -> 1.0 2 -> 0.0 3 -> 0.0)
(1,16,true,1 -> 1.0 2 -> 0.0 3 -> 0.0)
(6,4,false,)
(17,2,false,)
(20,3,true,1 -> 0.0 2 -> 0.0 3 -> 1.0)
(3,10,false,)
(9,5,true,1 -> 0.0 2 -> 0.0 3 -> 1.0)
(7,4,true,1 -> 0.0 2 -> 1.0 3 -> 0.0)
(12,1,false,)
(8,4,false,)
(18,2,false,)
(31,4,true,1 -> 0.0 2 -> 1.0 3 -> 0.0)
(26,3,true,1 -> 0.0 2 -> 1.0 3 -> 0.0)
(10,2,true,1 -> 1.0 2 -> 0.0 3 -> 0.0)
(2,9,true,1 -> 1.0 2 -> 0.0 3 -> 0.0)


In [37]:
val initialMsg = Map("1" -> 0.0, "2" -> 0.0, "3" -> 0.0 )
// val initialMsg = Map("1" -> 0.0, "2" -> 0.0)

val maxIter = 100
val vertex_count = new AtomicInteger()

val graphLP = finalGraph.pregel(initialMsg, maxIter, EdgeDirection.Either)(
    (id, vert, newLabels) => VertexProgram2(id, vert, newLabels),
    it => SendMsg(it),
    (a, b) => MergeMsg(a, b)
)

initialMsg: scala.collection.immutable.Map[String,Double] = Map(1 -> 0.0, 2 -> 0.0, 3 -> 0.0)
maxIter: Int = 100
vertex_count: java.util.concurrent.atomic.AtomicInteger = 0
graphLP: org.apache.spark.graphx.Graph[LPVertex,Int] = org.apache.spark.graphx.impl.GraphImpl@64da4f06


In [38]:
graphLP.vertices.foreach(v => println(v._1, v._2.degree, v._2.initialLabel, v._2.vertexLabels.mkString(" ")))

(13,2,true,1 -> 1.0 2 -> 0.0 3 -> 0.0)
(19,2,false,1 -> 0.05484161715566786 2 -> 0.2371409567985757 3 -> 0.7065618761905406)
(34,17,false,1 -> 0.05597349195396211 2 -> 0.24229817622203798 3 -> 0.7003639525005362)
(5,3,true,1 -> 0.0 2 -> 1.0 3 -> 0.0)
(15,2,false,1 -> 0.05484161715566786 2 -> 0.2371409567985757 3 -> 0.7065618761905406)
(4,6,true,1 -> 0.0 2 -> 0.0 3 -> 1.0)
(21,2,true,1 -> 0.0 2 -> 0.0 3 -> 1.0)
(16,2,false,1 -> 0.05484161715566786 2 -> 0.2371409567985757 3 -> 0.7065618761905406)
(22,2,true,1 -> 0.0 2 -> 1.0 3 -> 0.0)
(25,3,false,1 -> 0.06397607964859292 2 -> 0.26673906232994343 3 -> 0.667648960717103)
(28,4,true,1 -> 0.05914445589493983 2 -> 0.25340422325476836 3 -> 0.6859519111495134)
(29,3,true,1 -> 0.06139097665400267 2 -> 0.2607097769124235 3 -> 0.6765420049317887)
(11,3,false,1 -> 0.06599430422450676 2 -> 0.674207943538445 3 -> 0.2593688475904055)
(14,5,false,1 -> 0.05773038300977386 2 -> 0.2562729805520182 3 -> 0.6851968943900304)
(32,6,false,1 -> 0.06920401721870

In [39]:
// Get label estimate after algorithm
val m = graphLP.vertices.map({case(k,v) => (k.toLong, v.vertexLabels.maxBy{ case (key, value) => value }._1.toLong)})

m: org.apache.spark.rdd.RDD[(Long, Long)] = MapPartitionsRDD[3554] at map at <console>:37


In [40]:
//saving labels in file
// m.coalesce(1).saveAsTextFile("karate_labels")
// m.coalesce(1).saveAsTextFile("karate_2_labels")
m.coalesce(1).saveAsTextFile("karate_vertex_labels")